*6132-010402D Макарова Мария*

1. Загрузить датасет WINE.
2. Подготовить выборки: обучающую 70% и тестовую 30%.
3. Для указанных в задании классификаторов с использованием обучающей выборки выбрать 2 компоненты вектора признаков (2 признака), обеспечивающие наилучшее качество классификации.
4. Получить оценку качества классификации с использованием тестовой выборки.
5. Сравнить выбранные признаки для каждого из классификаторов и сделать вывод.
6. Название выбранные признаков для каждого классификатора, а также результаты классификации и вывод записать в качестве ответа в текстовое поле. В качестве ответа также разместить ipynb файл.

Классификаторы: {"SVM Sigmoid", "Quadratic Discriminant Analysis", "Nearest Neighbors 7"}


In [37]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

#Загрузить датасет WINE.
data = load_wine()
X, y = data.data, data.target
feature_names = data.feature_names

#Подготовить выборки: обучающую 70% и тестовую 30%.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

classifiers = {
    "SVM Sigmoid": SVC(kernel='sigmoid', random_state=42),
    "Quadratic Discriminant Analysis": QuadraticDiscriminantAnalysis(),
    "Nearest Neighbors": KNeighborsClassifier(n_neighbors=7)
}

#Для указанных в задании классификаторов с использованием обучающей выборки
#выбрать 2 компоненты вектора признаков (2 признака), обеспечивающие наилучшее качество классификации.
def select_best_features(classifier, X_train, y_train):
    best_score = 0
    best_features = None

    for feature_pair in combinations(range(X_train.shape[1]), 2):
        X_train_pair = X_train[:, feature_pair]

        scores = cross_val_score(classifier, X_train_pair, y_train, cv=5, scoring='accuracy')
        mean_score = scores.mean()

        if mean_score > best_score:
            best_score = mean_score
            best_features = feature_pair

    return best_features, best_score

results = {}

for clf_name, clf in classifiers.items():
    best_features, best_score = select_best_features(clf, X_train, y_train)

    X_train_best = X_train[:, best_features]
    X_test_best = X_test[:, best_features]
    clf.fit(X_train_best, y_train)

    #Получить оценку качества классификации с использованием тестовой выборки.
    y_pred = clf.predict(X_test_best)
    test_accuracy = accuracy_score(y_test, y_pred)

    results[clf_name] = {
        "Features": [feature_names[best_features[0]], feature_names[best_features[1]]],
        "Test Accuracy": test_accuracy
    }

for clf_name, result in results.items():
    print(f"Классификатор: {clf_name}")
    print(f"Выбранные признаки: {result['Features']}")
    print(f"Точность на тестовых данных: {result['Test Accuracy']:.4f}")
    print("-" * 40)

Классификатор: SVM Sigmoid
Выбранные признаки: ['alcohol', 'od280/od315_of_diluted_wines']
Точность на тестовых данных: 0.8704
----------------------------------------
Классификатор: Quadratic Discriminant Analysis
Выбранные признаки: ['flavanoids', 'color_intensity']
Точность на тестовых данных: 0.9259
----------------------------------------
Классификатор: Nearest Neighbors
Выбранные признаки: ['alcohol', 'flavanoids']
Точность на тестовых данных: 0.9074
----------------------------------------
